In this notebook you can train a student through DAgger for [FrankaKitchen](https://robotics.farama.org/envs/franka_kitchen/franka_kitchen/) environment. You should set:
- `seed`: For reproducibility of training runs

In [5]:
#@title Inizialization and import
import shutil
shutil.rmtree('DAgger4Robotics', ignore_errors=True)
!git clone "https://github.com/cybernetic-m/DAgger4Robotics.git"
!pip install gymnasium-robotics -q
!apt-get install -y xvfb ffmpeg -q
!pip install pyvirtualdisplay imageio -q
!pip install "minari[all]" -q

import gymnasium as gym
import gymnasium_robotics
gym.register_envs(gymnasium_robotics)
from gymnasium.wrappers import RecordVideo
import torch
import torch.nn as nn
import json
import os
import numpy as np
import minari # needed for dataset
import matplotlib.pyplot as plt

from DAgger4Robotics.model.NetworkInterface import NetworkInterface
from DAgger4Robotics.dagger.DAgger import DAgger
from DAgger4Robotics.utils.preprocess_dataset import preprocess_dataset
from DAgger4Robotics.simulator.Simulator import Simulator



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Reproducibility instructions
seed=42 #@param {type:"integer"}
np.random.seed(seed)                   # NumPy
torch.manual_seed(seed)                # PyTorch CPU
torch.cuda.manual_seed(seed)           # PyTorch GPU
torch.cuda.manual_seed_all(seed)       # All GPUs

torch.backends.cudnn.deterministic = True   # Deterministic behaviour
torch.backends.cudnn.benchmark = False      # Avoid non-deterministic optimizations

# Dimensions of the problem
state_dim = 20
action_dim = 9

Cloning into 'DAgger4Robotics'...
remote: Enumerating objects: 861, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 861 (delta 52), reused 18 (delta 4), pack-reused 768 (from 1)
Receiving objects: 100% (861/861), 163.21 MiB | 21.48 MiB/s, done.
Resolving deltas: 100% (330/330), done.
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.15).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
cuda


**THIS DATASET IS USED ONLY FOR EVALUATION**

In [2]:
#@title Loading and preprocess dataset
# Download the dataset
kitchen_dataset_type = 'complete'
validation_dataset = minari.load_dataset(f'D4RL/kitchen/{kitchen_dataset_type}-v2',download=True)
validation_dataset.set_seed(seed)  # Set a seed for reproducibility
print("Dataset loaded successfully!")
print(f'Total Episodes: {validation_dataset.total_episodes}')

# We preprocess the dataset in a way to extract a subset of observation needed only for the microwave task (open it)
# Then preprocess_dataset function will cut the complete kitchen dataset to the step in which the robot has completed the opening
# of the microwave (that is the first operation that it do in the env). In our cutted version of the dataset each obs, action is formed:
# Observation (20-dim):
#              - 0-6 joint angles
#              - 7-8 gripper joint translation value (LEFT-RIGHT pieces)
#              - 9-15 joint angular velocities
#              - 16-17 gripper joint linear velocity (LEFT-RIGHT pieces)
#              - 31  Rotation of the joint in the microwave door (angle)
#              - 52 Angular velocity of the microwave door joint
# Actions (9-dim):
#              - 0-6 joint angular velocities
#              - 7-8 gripper joint linear velocity (LEFT-RIGHT)
# You can access to the single episode and to the single step using 'microwave_dataset['observations/actions'][epi][step]'

#Using all the episodes to validate the new student
validation_microwave_dataset = preprocess_dataset(validation_dataset)

Dataset loaded successfully!
Total Episodes: 19


### Training Student Hyperparameters
This part of code runs and tests the DAgger algorithm. You must set
- `batch_size`: Number of samples state-action per batch
- `lr`: Learning rate for the optimizer
- `num_epochs`: Number of training epochs
- `rollouts_per_iteration`: Number of generated episodes in each iteration of DAgger
- `n_iterations`: Number of iteration of DAgger
- `betaMode`: The type of function that manages the decay of Beta. It can be `'inverse'`, `'linear'`, `'exponential'`
- `exponential_beta_k`: Exponent used in the beta function when betaMode is set to `'exponential'`
- `student_type`: It can be `'simple'` or `'deep'`, but generally `'simple'` for students

### Loading a different expert
If you want to use a different expert , you can load it manually by changing:
- `path_to_expert_model`: Insert here the complete path to the model provided by `Dagger4Robotics` folder
- `expert_type`: The architecture type of the loaded model (`"simple"` or `"deep"`)

In [3]:
#@title Run the DAgger Algorithm
loss_fn = nn.MSELoss()

# Hyperparameters for training the student
lr = 1e-3  #@param {type:"number"}
batch_size = 512 #@param {type:"integer"}
num_epochs = 5 #@param {type:"integer"}

# Number of generated episodes in each iteration of DAgger
rollouts_per_iteration=20 #@param {type:"integer"}

# Number of iteration of DAgger
n_iterations=20 #@param {type:"integer"}


# The type of function that manages the value of Beta. It can be 'inverse', 'linear', 'exponential'
betaMode = 'exponential' #@param ["inverse","linear","exponential"]

# Exponent used in the beta function when betaMode is set to 'exponential'
exponential_beta_k = 0.3 #@param {type:"number"}

# It can be 'simple' or 'deep, but generally 'simple' for students
student_type='simple' #@param ["simple","deep"]

# Initializing the student
net_wrapper = NetworkInterface(net_type=student_type,input_dim=state_dim,output_dim=action_dim)
net_wrapper.summary()
pi = net_wrapper.get_model().to(device)

# Loading pre-trained expert/teacher
expert_type='deep'
net_wrapper = NetworkInterface(net_type=expert_type,input_dim=state_dim,output_dim=action_dim)
net_wrapper.summary()
pi_star = net_wrapper.get_model().to(device)
pi_star.load_state_dict(torch.load('./DAgger4Robotics/experts_kitchen/deep/kitchen_complete/batch_size_64_lr_1e-3/expert_policy.pt', map_location=device))
pi_star.eval()

env = gym.make('FrankaKitchen-v1', tasks_to_complete=['microwave'])
optimizer = torch.optim.Adam(pi.parameters(), lr=lr)

# Initializing the DAgger algorithm
dagger = DAgger(env = env,
                env_type = 'kitchen',
                validationDataset= validation_microwave_dataset,
                studentPolicy = pi,
                expertPolicy = pi_star,
                optimizer=optimizer,
                loss_fn = loss_fn,
                batch_size = batch_size,
                num_epochs = num_epochs,
                betaMode = betaMode,
                device = device,
                rollouts_per_iteration=rollouts_per_iteration,
                exponential_beta_k = exponential_beta_k
                )

# Running the training
dagger.run(n_iterations=n_iterations)

SimplePolicyNet(
  (net): Sequential(
    (0): Linear(in_features=20, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=9, bias=True)
  )
)
DeepPolicyNet(
  (net): Sequential(
    (0): Linear(in_features=20, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=9, bias=True)
  )
)
Validation Dataset correctly loaded.
Obs Space Dim: 1058, Action Space Dim: 1058
Run DAgger algorithm...

--- ITERATION 1/20 | beta = 1.000 ---
Rollout... 
The Expert was selected 2251 times, while the student was selected 0 times
Collected data - Observations: 2251 Actions: 2251
Training Student Policy...
EPOCH 0:


Elements...: 100%|██████████| 5/5 [00:00<00:00, 43.24it/s]


TRAIN	 Loss: 0.06956767, RMSE: 0.21982442, MAE: 0.18712765, R2: -2.55954194
EPOCH 1:


Elements...: 100%|██████████| 5/5 [00:00<00:00, 414.80it/s]


TRAIN	 Loss: 0.05893051, RMSE: 0.20058390, MAE: 0.16608353, R2: -1.56348598
EPOCH 2:


...

TRAIN	 Loss: 0.00001435, RMSE: 0.00350988, MAE: 0.00264882, R2: 0.99864262
EPOCH 3:


Elements...: 100%|██████████| 94/94 [00:00<00:00, 400.55it/s]


TRAIN	 Loss: 0.00001345, RMSE: 0.00341902, MAE: 0.00257041, R2: 0.99869180
EPOCH 4:


Elements...: 100%|██████████| 94/94 [00:00<00:00, 387.88it/s]


TRAIN	 Loss: 0.00001328, RMSE: 0.00340410, MAE: 0.00256130, R2: 0.99869609
Model saved as student_policy_19.pt
Evaluating the best student...
student_policy_0.pt	 Loss: 0.02963963, RMSE: 0.15996771, MAE: 0.12446148, R2: 0.29197115
student_policy_1.pt	 Loss: 0.01622160, RMSE: 0.11347617, MAE: 0.08077682, R2: 0.54710692
student_policy_2.pt	 Loss: 0.01212965, RMSE: 0.09899024, MAE: 0.06923320, R2: 0.65899110
student_policy_3.pt	 Loss: 0.01005203, RMSE: 0.09263600, MAE: 0.06479467, R2: 0.69711530
student_policy_4.pt	 Loss: 0.00944234, RMSE: 0.09066804, MAE: 0.06328025, R2: 0.71609199
student_policy_5.pt	 Loss: 0.00921655, RMSE: 0.08883385, MAE: 0.06216289, R2: 0.72994620
student_policy_6.pt	 Loss: 0.00841510, RMSE: 0.08795118, MAE: 0.06162636, R2: 0.73700911
student_policy_7.pt	 Loss: 0.00949608, RMSE: 0.08686584, MAE: 0.06086705, R2: 0.74464816
student_policy_8.pt	 Loss: 0.00784993, RMSE: 0.08632495, MAE: 0.06044428, R2: 0.74970537
student_policy_9.pt	 Loss: 0.00850166, RMSE: 0.08601230, 

### Simulation
Use this section to run rollouts in the environment using the newly trained student network. You can customize the simulation with the following parameters:
- `idx`: <mark style="background-color:#ffcccc"><font color="red"><b>IMPORTANT</b></font></mark> 🚨 **Set this to the index of the best-performing model found in the training above** (Ex. "The better student Policy is the student_policy_12.pt" => Then you should put idx = 12)
- `render`: If True, displays on screen the rollouts. **Requires a GPU**
- `framerate_per_episode` : Controls how frequently frames are rendered. Only frames where `(frame_idx % framerate_per_episode == 0)` are shown
- `video_saving`: If True, saves the video of the episodes in `./new_videos` folder. In **Kitchen** environment, this critically increase computational time
- `n_episodes`: Number of episodes to simulate
- `robot_noise`: Magnitude of noise added to the robot’s proprioceptive variables (only for the **Kitchen** environment)

At the end of all the Rollouts, a mean_rewards.json file will be saved, and it will contain:
- The mean rewards for each episode
- `mean_of_means`: The overall mean reward across all episodes

### Using a Saved model

If you want to simulate one of our pretrained models, you can load it manually by changing:
- `path_to_model`: Insert here the complete path to the model provided by `Dagger4Robotics` folder
- `net_type`: The architecture type of the loaded model (`"simple"` or `"deep"`)

In [ ]:
#@title Parameters of the experiments { run: "auto" }
# Change this value to the best model found
idx=1 #@param {type:"integer"}
render = True #@param {type:"boolean"}
framerate_per_episode=5  #@param {type:"integer"}
video_saving = False #@param {type:"boolean"}
n_episodes = 1 #@param {type:"integer"}
robot_noise=0.01 #@param {type:"number"}

path_to_model=f"student_policy_{idx}.pt"
net_type=student_type

In [ ]:
sim = Simulator(
        env_mode='kitchen',
        net_type=net_type,
        path_to_model=path_to_model,
        n_episodes=n_episodes,
        render=render,
        framerate_per_episode=framerate_per_episode,
        video_saving=video_saving,
        robot_noise=robot_noise,
        device=device
    )
sim.run()